### Import libraries

In [135]:
import spacy
import random
from tqdm import tqdm
from nltk.tokenize import sent_tokenize
import pandas as pd
import re
import numpy as np

### Read data

In [136]:
data = pd.read_csv('data/tagging.csv')

In [137]:
data.head(5)

,Date,Company Name,Title,Corporate Action,Web-Link,Summary,Dividend,Dividend percentage (DP),Dividend Offer Value,Share Face Value (‘FV’),...,Divided Decision,Bonus/ Right Ratio,Bonus/Right Ex-date,Bonus Require,Bonus Provided,Bonus Decision,Meeting Scheduled Date,Intimation/Confirmed,Type of Meeting,KW
0,01-Nov-20,NACL Industries Ltd,Corporate Action-Board to consider Dividend,Dividend,https://www.bseindia.com/stockinfo/AnnPdfOpen....,In continuation to our letter dated 26th Octob...,dividend,NaN,NaN,NaN,...,consider,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31-Oct-20,BALKRISHNA INDUSTRIES LTD.,Board to Consider Q2 results and Second Interi...,Dividend,https://www.bseindia.com/stockinfo/AnnPdfOpen....,Balkrishna Industries Ltd has informed BSE tha...,dividend,NaN,NaN,NaN,...,to consider,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,30-Oct-20,Vaibhav Global Ltd,Board declares Second Interim Dividend,Dividend,https://www.bseindia.com/xml-data/corpfiling/A...,Vaibhav Global Ltd has informed BSE that the B...,Dividend,NaN,Rs. 5,Rs. 10,...,considered and approved,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,30-Oct,K.Z.LEASING & FINANCE LTD.,Board Meeting Intimation for Consideration Of ...,Meetings,https://www.bseindia.com/xml-data/corpfiling/A...,K.Z.LEASING &amp; FINANCE LTD.has informed BSE...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,05-11-2020,scheduled on,meeting of the Board of Directors,Board Meeting Intimation
4,30-Oct,KALYANI INVESTMENT COMPANY LTD.,Board Meeting Intimation for Consider And Appr...,Meetings,https://www.bseindia.com/xml-data/corpfiling/A...,KALYANI INVESTMENT COMPANY LTD.has informed BS...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,09-11-2020,scheduled on,meeting of the Board of Directors,NaN


### Data Processing

Convert the data into specific format of the training data which will be fed in the spacy model 

In [138]:
''' 
Getting the list of columns in dataframe 
'''
data.columns

Index(['Date', 'Company Name', 'Title', 'Corporate Action', 'Web-Link',
       'Summary', 'Dividend', 'Dividend percentage (DP)',
       'Dividend Offer Value', 'Share Face Value (‘FV’)', 'Dividend Ex- Date',
       'Dividend Announce Date', 'Divided Decision ', 'Bonus/ Right Ratio',
       'Bonus/Right Ex-date', 'Bonus Require ', 'Bonus Provided',
       'Bonus Decision', 'Meeting Scheduled Date', 'Intimation/Confirmed',
       'Type of Meeting', 'KW'],
      dtype='object')

In [139]:
'''
LABELS:

Div = Dividend
Percent = Dividend percentage (DP)
OfferV = Dividend Offer Value
FaceV = Share Face Value (‘FV’)
DivExDate = Dividend Ex- Date
AnnDate = Dividend Announce Date
Dec = Divided Decision
Ratio = Bonus/ Right Ratio
BonusExDate = Bonus/Right Ex-date
BonusReq = Bonus Require
BonusProv = Bonus Provided 
BonusDec = Bonus Decision
SDate = Meeting Scheduled Date
Int = Intimation/Confirmed
Type = Type of Meeting 

'''


'\nLABELS:\n\nDiv = Dividend\nPercent = Dividend percentage (DP)\nOfferV = Dividend Offer Value\nFaceV = Share Face Value (‘FV’)\nDivExDate = Dividend Ex- Date\nAnnDate = Dividend Announce Date\nDec = Divided Decision\nRatio = Bonus/ Right Ratio\nBonusExDate = Bonus/Right Ex-date\nBonusReq = Bonus Require\nBonusProv = Bonus Provided \nBonusDec = Bonus Decision\nSDate = Meeting Scheduled Date\nInt = Intimation/Confirmed\nType = Type of Meeting \n\n'

In [140]:
'''
Changing the format of the date to match the contents of summary
'''

date = list(data['Meeting Scheduled Date'])
for i in range(len(date)):
    if not (date[i] is np.nan):
        date[i] = date[i].replace('-','/') 
        
data['Meeting Scheduled Date'] = date

In [141]:
'''
list of the enities columns of data = columns[]
list of the labels given to these entities = labels[]
'''
columns = ['Dividend', 'Dividend percentage (DP)','Dividend Offer Value', 'Share Face Value (‘FV’)', 'Dividend Ex- Date', 'Dividend Announce Date', 'Divided Decision ', 'Bonus/ Right Ratio', 'Bonus/Right Ex-date', 'Bonus Require ', 'Bonus Provided','Bonus Decision', 'Meeting Scheduled Date', 'Intimation/Confirmed','Type of Meeting']
labels = ['Div', 'Percent', 'OfferV', 'FaceV', 'DivExDate', 'AnnDate', 'Dec', 'Ratio', 'BonusExDate', 'BonusReq', 'BonusProv', 'BonusDec', 'SDate', 'Int', 'Type']

In [142]:
'''
list of summaries (column in data)
'''
summaries = list(data['Summary'])

In [143]:
'''
drop the rest of the columns other than entities 
'''
data = data[columns]
data.sample(n=5)

,Dividend,Dividend percentage (DP),Dividend Offer Value,Share Face Value (‘FV’),Dividend Ex- Date,Dividend Announce Date,Divided Decision,Bonus/ Right Ratio,Bonus/Right Ex-date,Bonus Require,Bonus Provided,Bonus Decision,Meeting Scheduled Date,Intimation/Confirmed,Type of Meeting
57,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,06/11/2020,scheduled on,meeting of the Board of Directors
123,dividend,40%,Rs. 0.80,Rs. 2,NaN,"June 30, 2020",recommended,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
152,dividend,100%,Rs. 10,Rs. 10,NaN,NaN,Recommended,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,Dividend,NaN,NaN,NaN,NaN,NaN,consider,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09/11/2020,scheduled on,meeting of the Board of Directors


In [144]:
'''
converting each row into a list and then appending it to a final 2D list
'''

tags=[] #2D List

for i in range(len(data)):
    temp = [str(x) for x in data.iloc[i].tolist()]
    tags.append(temp)
    temp=[]
    
#print(tags)


In [145]:
'''
convert the data into specific format of the training data
TRAIN_DATA = [
    ("Who is Shaka Khan?", {"entities": [(7, 17, "PERSON")]}),
    ("I like London and Berlin.", {"entities": [(7, 13, "LOC"), (18, 24, "LOC")]}),
]
'''

train_data = []

count = 0;
for summary,tag in zip(summaries, tags):
    temp=[]
    for index in range(len(tag)):
        entity = tag[index]
        if entity != 'nan':
            pos = summary.find(entity)
            if(pos != -1):
                entity_set = (pos, pos+len(entity), labels[index]) #(350, 358, 'Summary')
                temp.append(entity_set) #list of entitity (described above)
        
    #print(temp)
    
    ''' temp = [(350, 358, 'Summary'), (None, None, 'Dividend'), (None, None, 'Dividend percentage (DP)'), (None, None, 'Dividend Offer Value'), (None, None, 'Share Face Value (‘FV’)'), (None, None, 'Dividend Ex- Date'), (145, 153, 'Dividend Announce Date'), (None, None, 'Divided Decision'), (None, None, 'Bonus/ Right Ratio'), (None, None, 'Bonus/Right Ex-date'), (None, None, 'Bonus Require '), (None, None, 'Bonus Provided'), (None, None, 'Bonus Decision'), (None, None, 'Meeting Scheduled Date'), (None, None, 'Intimation/Confirmed')]'''
    
    inner_dict = {'entities': temp}
    inner_set = (summary, inner_dict)
    train_data.append(inner_set)
    
    count+=1

In [158]:
'''
training data
'''
print(train_data[24])

('Pursuant to Regulation 42 of the SEBI (Listing Obligations and Disclosure Requirements) Regulations, 2015, we wish to inform you that the Register of Members and Share Transfer Register shall remain closed from 9th December, 2020 to 15th December, 2020 (both days inclusive) for the purpose of 32nd Annual General Meeting of the Company which is scheduled to be held on Tuesday, the 15th December, 2020 through Video Conferencing (VC) or Other Audio-Visual Means (OAVM) in compliance with Circular dated May 5, 2020 read with Circulars dated April 8, 2020 and April 13, 2020 issued by the Ministry of Corporate Affairs.&lt;BR&gt; &lt;BR&gt; Kindly take the above information on record and acknowledge receipt of the same.', {'entities': [(233, 252, 'SDate'), (346, 369, 'Int'), (299, 321, 'Type')]})


In [159]:
for i in range(len(train_data)):
    if(train_data[i][0] == 'Recommended a dividend of Rs. 10/- per Ordinary Share of Rs. 10/- each (100%) and Rs. 2.504 per partly paid Ordinary Share of Rs. 10/- each (paid-up Rs. 2.504 per share) (100%) to the shareholders for the Financial Year ended March 31, 2020.'):
        print(i)
        print(train_data[i][1])

152
{'entities': [(14, 22, 'Div'), (72, 76, 'Percent'), (26, 32, 'OfferV'), (26, 32, 'FaceV'), (0, 11, 'Dec')]}


In [147]:
exception_summary=[]
exception_annotations=[]

In [148]:
l=[]
s=()
i=0
index=[]
for set_temp in train_data:
    annotations = set_temp[1]['entities']
    l=[]
    for entity in annotations:
        l.append(entity[0])
        l.append(entity[1])
    if( len(l) > len(set(l)) ):
        print(set_temp[0])
        print(annotations)
        print("-------------------------------------------")
        exception_summary.append(set_temp[0])
        exception_annotations.append(set_temp[1])
        index.append(i)
    i+=1

Thyrocare Technologies Ltd has informed BSE that the Board of Directors of the Company at its meeting held on October 28, 2020, inter alia, have approved payment of interim dividend for the financial year 2020-21, at the rate of Rs. 10/- (Rupees Ten only) per share (100% of the face value of Rs. 10/? each).
[(173, 181, 'Div'), (267, 271, 'Percent'), (229, 235, 'OfferV'), (229, 235, 'FaceV'), (145, 153, 'Dec')]
-------------------------------------------
The Board declared an interim dividend of Rs.10 (100%) per equity share of Rs.10 each. &lt;BR&gt; &lt;BR&gt; As intimated vide our letter dated October 13, 2020, Thursday, October 29, 2020 is fixed as the Record Date to ascertain the list of shareholders eligible for the aforesaid interim dividend to be paid on or before November 19, 2020.&lt;BR&gt;
[(30, 38, 'Div'), (49, 53, 'Percent'), (42, 47, 'OfferV'), (42, 47, 'FaceV'), (323, 340, 'DivExDate'), (10, 18, 'Dec')]
-------------------------------------------
We would like to inform yo

In [149]:
index

[18, 102, 113, 129, 135, 143, 152, 180, 215, 217, 242, 248, 253, 254, 255]

In [206]:
x=0
for i in index:
    train_data[i][1]['entities'] = ast.literal_eval(exception['Annotations'][x][13:-1])
    x+=1

In [209]:
t = pd.DataFrame(columns=['Summary', 'Annotations'])

In [210]:
s=[]
a=[]
for set_temp in train_data:
    s.append(set_temp[0])
    a.append(set_temp[1])

In [211]:
t['Summary']=s
t['Annotations']=a

In [212]:
t.to_csv('data/training_data.csv')

### Load NER (Name Entity Recognition)spacy model

In [14]:
nlp = spacy.blank('en')

In [15]:
'''
creating and adding "ner" pipeline if not exist
'''

if "ner" not in nlp.pipe_names:
    ner = nlp.create_pipe("ner")
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe("ner")
    


In [16]:
'''
fixing number of iterations model has to do in training
At each iteration, the training data is shuffled to ensure the model doesn’t make any generalizations based on the order of examples.
'''

n_iter=100

### Add the above made labels in the ner (name entity recognition) pipe

In [17]:
for _, annotations in train_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

In [18]:
'''
get names of other pipes to disable them during training
'''
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

### Train the model

In [19]:
count=0
exception_summary=[]
exception_annotations=[]
with nlp.disable_pipes(*other_pipes):  # only train NER pipeline
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(train_data)
        losses = {}
        try:
            for text, annotations in tqdm(train_data):
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.5,  # dropout makes it harder for the model to memorize the training data.
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
        except:
            count+=1
            exception_summary.append(text)
            exception_annotations.append(annotations)   
        print(losses)

C:\Users\Vanshika Sharma\Anaconda3\lib\site-packages\spacy\language.py:639: UserWarning: [W033] Training a new parser or NER using a model with no lexeme normalization table. This may degrade the performance of the model to some degree. If this is intentional or the language you're using doesn't have a normalization table, please ignore this warning. If this is surprising, make sure you have the spacy-lookups-data package installed. The languages with lexeme normalization tables are currently: da, de, el, en, id, lb, pt, ru, sr, ta, th.
  **kwargs
  0%|          | 1/300 [00:00<01:39,  3.02it/s]C:\Users\Vanshika Sharma\Anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Recommended a final dividend of Re 0.01/- per Equi..." with entities "[(20, 28, 'Div'), (64, 66, 'Percent'), (32, 39, 'O...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be 

{'ner': 1662.6420784432219}
{'ner': 5.883854251024134}


C:\Users\Vanshika Sharma\Anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "We would like to inform you that the Board of Dire..." with entities "[(383, 391, 'Div'), (107, 114, 'OfferV'), (140, 14...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
  1%|▏         | 4/300 [00:00<00:21, 13.83it/s]C:\Users\Vanshika Sharma\Anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Navin Fluorine International Ltd has informed BSE ..." with entities "[(170, 178, 'Div'), (226, 230, 'Percent'), (182, 1...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
  3%|▎         | 8/300 [00:00<00:21, 13.5

{'ner': 110.2978538758804}


  2%|▏         | 5/300 [00:00<00:24, 12.21it/s]C:\Users\Vanshika Sharma\Anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Pursuant to Regulation 30 of SEBI (Listing Obligat..." with entities "[(145, 153, 'Div'), (193, 196, 'Percent'), (157, 1...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **gold)
  3%|▎         | 9/300 [00:00<00:22, 13.04it/s]C:\Users\Vanshika Sharma\Anaconda3\lib\site-packages\spacy\language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "Recommendation of final dividend subject to approv..." with entities "[(24, 32, 'Div'), (66, 69, 'Percent'), (75, 83, 'O...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  gold = GoldParse(doc, **

...

{'ner': 8.006540469188016}


  1%|          | 2/300 [00:00<00:20, 14.32it/s]

{'ner': 11.839158513420703}


  8%|▊         | 24/300 [00:01<00:21, 13.08it/s]

{'ner': 115.31049517375882}


In [20]:
print(count)

100


### Save/Download the model 

In [21]:
nlp.to_disk(r'ner_spacy_model')

### Load the model for testing

In [22]:
nlp2 = spacy.load(r'ner_spacy_model')

### Test model

In [23]:
text = input()

 KALYANI INVESTMENT COMPANY LTD.has informed BSE that the meeting of the Board of Directors of the Company is scheduled on 09/11/2020 ,inter alia, to consider and approve Pursuant to Regulation 29 of SEBI (Listing Obligations and Disclosure Requirements) Regulations, 2015, this is to inform you that the meeting of the Board of Directors of the Company is scheduled to be held on Monday, November 9, 2020, inter alia, to consider and approve the Unaudited (Standalone &amp; Consolidated) Financial Results for the quarter and half year ended September 30, 2020. &lt;BR&gt; &lt;BR&gt; The Trading Window for dealing in securities of the Company, for all Designated and Connected Persons including their immediate relatives, shall be re-opened from Thursday, November 12, 2020. &lt;BR&gt;


In [24]:
'''
tokenize the input text
'''
sentences = sent_tokenize(text)
print(sentences)

['KALYANI INVESTMENT COMPANY LTD.has informed BSE that the meeting of the Board of Directors of the Company is scheduled on 09/11/2020 ,inter alia, to consider and approve Pursuant to Regulation 29 of SEBI (Listing Obligations and Disclosure Requirements) Regulations, 2015, this is to inform you that the meeting of the Board of Directors of the Company is scheduled to be held on Monday, November 9, 2020, inter alia, to consider and approve the Unaudited (Standalone &amp; Consolidated) Financial Results for the quarter and half year ended September 30, 2020.', '&lt;BR&gt; &lt;BR&gt; The Trading Window for dealing in securities of the Company, for all Designated and Connected Persons including their immediate relatives, shall be re-opened from Thursday, November 12, 2020.', '&lt;BR&gt;']


In [25]:
'''
test the model
'''
for sentence in sentences:
    doc2 = nlp2(sentence)
    #print(doc2)
    for ent in doc2.ents:
        print(ent.label_, "->", ent.text)

Type -> meeting of the Board of Directors
Int -> scheduled on
SDate -> 09/11/2020


In [26]:
doc2 = nlp2(text)
    #print(doc2)
for ent in doc2.ents:
    print(ent.label_, "->", ent.text)

Type -> meeting of the Board of Directors
Int -> scheduled on
SDate -> 09/11/2020


In [27]:
temp_count=0
for i,j in zip(exception_summary,exception_annotations):
    if(temp_count==5):
        break
    print(i)
    print(j)
    print("******************************************************")
    temp_count+=1

Force Motors Ltd has informed BSE that the Board of Directors of the Company at its meeting held on June 29, 2020, inter alia, has recommended a dividend of Rs. 10/- (Rupees ten only) per equity share of Rs. 10/- each, for the Financial Year ended March 31, 2020.
{'entities': [(145, 153, 'Div'), (157, 163, 'OfferV'), (157, 163, 'FaceV'), (100, 113, 'AnnDate'), (131, 142, 'Dec')]}
******************************************************
The Board of Directors of the Company has also recommended, subject to the approval of the Shareholders at the Annual General Meeting dividend as under: Dividend @ 125% i.e Rs. 2.50 /- per Equity Share of the face value of Rs. 2/- each for the financial year ended 31° March, 2020. 
{'entities': [(134, 142, 'Div'), (164, 168, 'Percent'), (173, 181, 'OfferV'), (173, 178, 'FaceV'), (47, 58, 'Dec')]}
******************************************************
This is to inform you that the Board of Directors at its Meeting held today has recommended Bonus Issue of 

In [28]:
import pandas as pd
df = pd.DataFrame(columns=['Summary','annotations'])
df['Summary']=exception_summary
df['annotations']=exception_annotations
df.to_csv("exception.csv",index=False)


In [29]:
test="rs 10 and rs 10 writing shit rifafas now and rs 10"

In [30]:
def findOccurrences(test_str, test_sub):
    len_test=len(test_sub)
    res = [i for i in range(len(test_str)) if test_str.startswith(test_sub, i)] 
    for _ in res:
        print(_," ",_+len_test)
findOccurrences(test,'rs 10')

0   5
10   15
45   50


In [79]:
def findOccurrences(test_str, test_sub):
    len_test=len(test_sub)
    res = [i for i in range(len(test_str)) if test_str.startswith(test_sub, i)] 
    count=0
    ret=[]
    for _ in res:
        count+=1
        ret.append([_,_+len_test])
        #print(_," ",_+len_test)    
    return(count,ret)

new=[]
d = dict()

temp_count=1
for i,j in zip(exception_summary,exception_annotations):
    print(temp_count)
    print(i,'\n')
    print(j,'\n')
    
    #print(j['entities'])
    for k in j['entities']:
        print(k ," --->   ",i[k[0]:k[1]])
        
        cnt,ret=findOccurrences(i,i[k[0]:k[1]])
        if(cnt>1):
            for _ in ret:
                print(_[0],",",_[1])
            #print('\n')
            
        #print('\n')
    
    temp_count+=1
    print("*******************************************************************************************")
    
    
    s = (i)
    

1
Essel Propack Ltd has informed BSE that Board of Directors of the Company at its Meeting held on April 26, 2014 has recommended Bonus Issue of Equity Shares in the ratio of 1 (One) Equity Share of Rs. 2/- each for every 1 (One) Equity Share of Rs. 2/- each held by the shareholders of the Company as on the record date/date fixed for this purpose, subject to the approval of the Members. 

{'entities': [(173, 180, 'BonusReq'), (173, 180, 'BonusProv'), (116, 127, 'BonusDec')]} 

(173, 180, 'BonusReq')  --->    1 (One)
173 , 180
220 , 227
(173, 180, 'BonusProv')  --->    1 (One)
173 , 180
220 , 227
(116, 127, 'BonusDec')  --->    recommended
*******************************************************************************************
2
We would like to inform you that at the Board Meeting held today, the Directors have declared a second interim dividend of Rs. 12 per Equity Share of Rs. 1 each of the Company.&lt;BR&gt; &lt;BR&gt; The second interim dividend shall be paid on Tuesday, Novembe

In [80]:
exception_annotations

[{'entities': [(173, 180, 'BonusReq'),
   (173, 180, 'BonusProv'),
   (116, 127, 'BonusDec')]},
 {'entities': [(111, 119, 'Div'),
   (123, 129, 'OfferV'),
   (123, 128, 'FaceV'),
   (252, 268, 'DivExDate'),
   (85, 93, 'Dec')]},
 {'entities': [(597, 604, 'BonusReq'),
   (597, 604, 'BonusProv'),
   (235, 246, 'BonusDec')]},
 {'entities': [(134, 142, 'Div'),
   (164, 168, 'Percent'),
   (173, 181, 'OfferV'),
   (173, 178, 'FaceV'),
   (47, 58, 'Dec')]},
 {'entities': [(145, 153, 'Div'),
   (157, 163, 'OfferV'),
   (157, 163, 'FaceV'),
   (100, 113, 'AnnDate'),
   (131, 142, 'Dec')]},
 {'entities': [(443, 451, 'Div'),
   (753, 756, 'Percent'),
   (556, 560, 'OfferV'),
   (556, 561, 'FaceV'),
   (429, 440, 'Dec')]},
 {'entities': [(14, 22, 'Div'),
   (72, 76, 'Percent'),
   (26, 32, 'OfferV'),
   (26, 32, 'FaceV'),
   (0, 11, 'Dec')]},
 {'entities': [(385, 392, 'BonusReq'),
   (385, 392, 'BonusProv'),
   (137, 147, 'BonusDec')]},
 {'entities': [(97, 100, 'BonusReq'),
   (97, 100, 'BonusPro

In [81]:
exception_summary

['Essel Propack Ltd has informed BSE that Board of Directors of the Company at its Meeting held on April 26, 2014 has recommended Bonus Issue of Equity Shares in the ratio of 1 (One) Equity Share of Rs. 2/- each for every 1 (One) Equity Share of Rs. 2/- each held by the shareholders of the Company as on the record date/date fixed for this purpose, subject to the approval of the Members.',
 'We would like to inform you that at the Board Meeting held today, the Directors have declared a second interim dividend of Rs. 12 per Equity Share of Rs. 1 each of the Company.&lt;BR&gt; &lt;BR&gt; The second interim dividend shall be paid on Tuesday, November 3, 2020 to the equity shareholders of the Company, whose names appear on the Register of Members of the Company or in the records of the Depositories as beneficial owners of the shares as on Thursday, October 15, 2020 which is the Record Date fixed for the purpose.',
 'LG Balakrishnan &amp; Bros Ltd has informed BSE that the Board of Directors

In [213]:
l

"[(173, 181, 'Div'), (267, 271, 'Percent'), (229, 235, 'OfferV'), (293, 299, 'FaceV'), (145, 153, 'Dec')]"